Extracting the Dataset

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"srirupinpotula","key":"ca97d95d40429d3e4f783eaa67a5c941"}'}

In [2]:
!pip install kaggle


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!kaggle datasets download -d jonathanoheix/face-expression-recognition-dataset

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [4]:
!mv .kaggle /root/

mv: cannot stat '.kaggle': No such file or directory


In [5]:
mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [6]:
mv kaggle.json /root/.kaggle

In [7]:
!kaggle datasets download -d jonathanoheix/face-expression-recognition-dataset

 95% 115M/121M [00:00<00:00, 186MB/s] 
100% 121M/121M [00:00<00:00, 164MB/s]


In [8]:
ls /content/face-expression-recognition-dataset.zip

/content/face-expression-recognition-dataset.zip


In [10]:
!unzip /content/face-expression-recognition-dataset.zip 

Streaming output truncated to the last 5000 lines.
  inflating: images/images/train/sad/11739.jpg  
  inflating: images/images/train/sad/1175.jpg  
  inflating: images/images/train/sad/11752.jpg  
  inflating: images/images/train/sad/11753.jpg  
  inflating: images/images/train/sad/11777.jpg  
  inflating: images/images/train/sad/11778.jpg  
  inflating: images/images/train/sad/11787.jpg  
  inflating: images/images/train/sad/11792.jpg  
  inflating: images/images/train/sad/11800.jpg  
  inflating: images/images/train/sad/11803.jpg  
  inflating: images/images/train/sad/11815.jpg  
  inflating: images/images/train/sad/11817.jpg  
  inflating: images/images/train/sad/11821.jpg  
  inflating: images/images/train/sad/11822.jpg  
  inflating: images/images/train/sad/11826.jpg  
  inflating: images/images/train/sad/11827.jpg  
  inflating: images/images/train/sad/11839.jpg  
  inflating: images/images/train/sad/1184.jpg  
  inflating: images/images/train/sad/11852.jpg  
  inflating: images/

In [11]:
ls

drive/  face-expression-recognition-dataset.zip  images/  sample_data/


In [12]:
ls images

images/


In [13]:
cd images 

/content/images


In [14]:
pwd


'/content/images'

In [15]:
ls 

images/


In [17]:
cd ../sample_data && ls

[Errno 2] No such file or directory: '../sample_data && ls'
/content/images


In [18]:
cd ../


/content


In [19]:
rm -rf images

In [20]:
ls


drive/  face-expression-recognition-dataset.zip  sample_data/


In [21]:
!unzip face-expression-recognition-dataset.zip


Streaming output truncated to the last 5000 lines.
  inflating: images/validation/fear/8797.jpg  
  inflating: images/validation/fear/8818.jpg  
  inflating: images/validation/fear/886.jpg  
  inflating: images/validation/fear/9037.jpg  
  inflating: images/validation/fear/9040.jpg  
  inflating: images/validation/fear/9101.jpg  
  inflating: images/validation/fear/911.jpg  
  inflating: images/validation/fear/9179.jpg  
  inflating: images/validation/fear/9205.jpg  
  inflating: images/validation/fear/9232.jpg  
  inflating: images/validation/fear/9251.jpg  
  inflating: images/validation/fear/9261.jpg  
  inflating: images/validation/fear/9281.jpg  
  inflating: images/validation/fear/9302.jpg  
  inflating: images/validation/fear/9333.jpg  
  inflating: images/validation/fear/9369.jpg  
  inflating: images/validation/fear/9370.jpg  
  inflating: images/validation/fear/9474.jpg  
  inflating: images/validation/fear/949.jpg  
  inflating: images/validation/fear/9602.jpg  
  inflating:

In [22]:
ls images 

images/  train/  validation/


Dataset Extraction for various images

In [25]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)


In [33]:
train_generator = train_datagen.flow_from_directory(
        '/content/images/train',  
        target_size=(48, 48),  
        class_mode='categorical'
)

Found 28821 images belonging to 7 classes.


In [34]:
validation_generator = validation_datagen.flow_from_directory(
        '/content/images/validation',  
        target_size=(48, 48),  
        class_mode='categorical')

Found 7066 images belonging to 7 classes.


Model Definition

In [43]:
import tensorflow as tf

model = tf.keras.models.Sequential([

    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(48, 48, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(7, activation='softmax')
])
   

In [44]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_22 (Conv2D)          (None, 46, 46, 16)        448       
                                                                 
 max_pooling2d_22 (MaxPoolin  (None, 23, 23, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_23 (Conv2D)          (None, 21, 21, 32)        4640      
                                                                 
 max_pooling2d_23 (MaxPoolin  (None, 10, 10, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_24 (Conv2D)          (None, 8, 8, 64)          18496     
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 4, 4, 64)        

**Compile the Model**

In [45]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

In [46]:
history = model.fit(
      train_generator, 
      epochs=15,
      verbose=1,
      validation_data = validation_generator
    )

Epoch 1/15
901/901 [==============================] - 50s 55ms/step - loss: 0.3675 - accuracy: 0.3267 - val_loss: 0.3285 - val_accuracy: 0.4298
Epoch 2/15
901/901 [==============================] - 49s 55ms/step - loss: 0.3143 - accuracy: 0.4549 - val_loss: 0.3001 - val_accuracy: 0.4878
Epoch 3/15
901/901 [==============================] - 50s 56ms/step - loss: 0.2875 - accuracy: 0.5149 - val_loss: 0.2929 - val_accuracy: 0.5122
Epoch 4/15
901/901 [==============================] - 50s 56ms/step - loss: 0.2684 - accuracy: 0.5555 - val_loss: 0.2778 - val_accuracy: 0.5342
Epoch 5/15
901/901 [==============================] - 49s 54ms/step - loss: 0.2516 - accuracy: 0.5928 - val_loss: 0.2779 - val_accuracy: 0.5481
Epoch 6/15
901/901 [==============================] - 49s 55ms/step - loss: 0.2340 - accuracy: 0.6272 - val_loss: 0.2728 - val_accuracy: 0.5634
Epoch 7/15
901/901 [==============================] - 55s 61ms/step - loss: 0.2151 - accuracy: 0.6687 - val_loss: 0.2819 - val_accuracy: